## lightGBM(Reg) + BoW(tf-idf) + sudachi(C)

In [1]:
"""from sklearn.metrics import cohen_kappa_score
	y_true = [2, 0, 2, 2, 0, 1]
	y_pred = [0, 0, 2, 2, 0, 2]
	cohen_kappa_score(y_true, y_pred)
	# weights="quadratic"でおk
	>>> 0.4285714285714286"""

'from sklearn.metrics import cohen_kappa_score\n\ty_true = [2, 0, 2, 2, 0, 1]\n\ty_pred = [0, 0, 2, 2, 0, 2]\n\tcohen_kappa_score(y_true, y_pred)\n\t# weights="quadratic"でおk\n\t>>> 0.4285714285714286'

In [35]:
# ライブラリの導入

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import cohen_kappa_score

def adapt_labels(labels):
  max_label = np.max(labels) # 2 or 4

  if max_label == 2:
    labels += 2
  elif max_label == 4:
    labels -= 2
  else:
    print('Exception error! please check with np.unique(labels).')
  return labels

In [33]:
# u1 = np.unique(y_preds)
# print(u1)
# adapt_labels(y_preds)
# u2 = np.unique(y_preds)
# print(u2)

[0. 1. 2. 3. 4.]
[-2. -1.  0.  1.  2.]


In [19]:
u1 = np.unique(y_preds)
print(u1)
adapt_labels(y_preds,reverse=False)
u2 = np.unique(y_preds)
print(u2)

[-2. -1.  0.  1.  2.]
[0. 1. 2. 3. 4.]


In [3]:
#ファイル
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/grad_comp

Mounted at /content/drive
/content/drive/MyDrive/grad_comp


## load data

In [4]:
# preprocess path
prepro = 'sudachi/C'
n_features = 10000

In [5]:
# setup data

f = open('preprocess/' + prepro + '/text.prep_train.txt', 'r')
train_data = f.read()
train_data = train_data.split('\n')
del train_data[30000]
#x_train = np.zeros((30000, len(res)))

f = open('preprocess/' + prepro + '/text.prep_test.txt', 'r')
test_data = f.read()
test_data = test_data.split('\n')
del test_data[2500]
#x_test = np.zeros((2500, len(res)))

f = open('preprocess/' + prepro + '/text.prep_dev.txt', 'r')
dev_data = f.read()
dev_data = dev_data.split('\n')
del dev_data[2500]
#x_dev = np.zeros((2500, len(res)))

# label
y_train = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.train.txt')
y_dev = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.dev.txt')
y_train += 2
y_dev += 2
u, counts = np.unique(y_train, return_counts=True)
print(u)
print(counts)

[0. 1. 2. 3. 4.]
[3543 5593 9227 7760 3877]


## TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = n_features)
X_train = vectorizer.fit_transform(train_data)
X_dev = vectorizer.transform(dev_data)
X_test = vectorizer.transform(test_data)

In [7]:
X_train.shape

(30000, 10000)

QWK = 0.041  C = 0.01
QWK = 0.210  C = 0.1
QWK = 0.334  C = 1
QWK = 0.324  C = 10
QWK = 0.290  C = 100
最適なハイパーパラメタは C = 1

In [8]:
best_qwk = 0
best_c = 1
for c in [1, 5,10, 50]:
    model = LogisticRegression(C=c, random_state=0,max_iter=1500)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_dev)
    qwk = cohen_kappa_score(y_dev,y_pred, weights='quadratic')
    if qwk > best_qwk:
        best_qwk = qwk
        best_c = c
    print("QWK = %.3f  C = %s" % (qwk, str(c)))
print("最適なハイパーパラメタは C = %s" % str(best_c))


model = LogisticRegression(C=best_c, random_state=0,max_iter=1500)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

y_preds -= 2
with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')

QWK = 0.330  C = 1
QWK = 0.338  C = 5
QWK = 0.324  C = 10
QWK = 0.299  C = 50
最適なハイパーパラメタは C = 5


-2 origin(nagisa)
QWK = 0.329  C = 1
QWK = 0.339  C = 5
QWK = 0.333  C = 10
QWK = 0.294  C = 50
最適なハイパーパラメタは C = 5

0 origin(顔文字あり)


In [ ]:
c_best = 5

In [ ]:
model = LogisticRegression(C=best_c, random_state=0,max_iter=1500)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')

In [ ]:
y_preds

array([0., 0., 0., ..., 1., 1., 1.])

In [14]:
print(f'最適なハイパラ C = {str(best_c)} : {prepro}')

最適なハイパラ C = 5 : sudachi/C


In [ ]:
best_qwk = 0
best_c = 1
for c in [0.01, 0.1, 1, 10, 100]:
    model = SVC(kernel='rbf',C=c, random_state=0)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_dev)
    qwk = cohen_kappa_score(y_dev,y_pred, weights='quadratic')
    if qwk > best_qwk:
        best_qwk = qwk
        best_c = c
    print("QWK = %.3f   C = %s" % (qwk, str(c)))

# add prepro path
print(f'最適なハイパラ C = {str(best_c)} : {prepro}')

model = SVC(kernel='rbf',C=best_c, random_state=0)
model.fit(X_train, y_train)
y_preds = model.predict(X_dev)
y_preds -= 2

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')

QWK = 0.000   C = 0.01
QWK = 0.021   C = 0.1
QWK = 0.311   C = 1
QWK = 0.372   C = 10
QWK = 0.371   C = 100
最適なハイパーパラメータは C = 1


In [ ]:
best_c = 10
model = SVC(kernel='rbf',C=best_c, random_state=0)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')

In [ ]:
y_preds = model.predict(X_test)

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')